In [2]:
import random
import projetannuel

from lib_gridfs import load_posters_with_size, stock_metrics
imgs, genres = load_posters_with_size((15, 10), genres=['Horreur', 'Animation', 'Histoire'])
imgs_as_lists = [img.tolist() for img in imgs]
import numpy as np

imgs_as_lists = np.array(imgs_as_lists)/255.0

data = list(zip(imgs_as_lists, genres))
# filtered_data = [(img, genre) for img, genre in data if genre in ['Animation', 'Horreur']]
random.shuffle(data)
imgs_shuffled = [movie[0] for movie in data]
genres_shuffled = [movie[1] for movie in data]
# genres_shuffled_binarygenres_shuffled_binary = []
# for g in genres_shuffled:
#     .append(0 if g == "Horreur" else 1)
lim = int(len(data) * 0.8)
imgs_as_lists_train = imgs_shuffled[:lim]
genres_train = genres_shuffled[:lim]
imgs_as_lists_test = imgs_shuffled[lim:]
genres_test = genres_shuffled[lim:]
from scipy.spatial.distance import pdist
import numpy as np

def median_pairwise_squared_distance(X):
    dists = pdist(X, metric='sqeuclidean')
    return np.median(dists)

gamma = 1.0 / (2 * median_pairwise_squared_distance(imgs_shuffled))
gamma

📦 Found 2962 images of size 15x10


📥 Loading resized images: 100%|██████████| 2962/2962 [00:02<00:00, 998.48it/s] 


np.float64(0.008948757371629201)

In [4]:
from projetannuel import SVM, SVMKernelType, SoftMargin

model = SVM(SVMKernelType.Linear(),SoftMargin.Hard())
model.train(imgs_as_lists_train,genres_train)
Y_pred = [model.predict([x])[0] for x in imgs_as_lists_test]
print(f"Accuracy Score : {projetannuel.accuracy_score(genres_test, Y_pred)}")

TypeError: argument 'y_py': must be real number, not str

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from projetannuel import SVMOvO, SVMKernelType, SoftMargin, SVMOvO

# 1. Generate synthetic multiclass data (3 classes, 150 samples, 10 features)
X, y = make_classification(
    n_samples=3000,
    n_features=450,
    n_informative=6,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

# 2. Encode y to ensure integer labels 0, 1, 2 (not strictly needed here, but for consistency)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# 4. Convert to Python lists
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()

# 5. Choose kernel and margin
gamma = 1 / X.shape[1]  # ~1 / n_features
kernel = SVMKernelType.rbf(gamma)
margin = SoftMargin.soft(1.0)

# 6. Initialize and train model
model = SVMOvO(kernel, margin)
model.train(X_train_list, y_train_list)

# 7. Predict
y_pred = model.predict(X_test_list)

# 8. Evaluate
acc = accuracy_score(y_test_list, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")
print(f"🧪 True labels: {y_test_list}")
print(f"📈 Predicted:  {y_pred}")


OSQP status   = Solved(Solution { x: [4.780340056230779e-23, -2.52665631098339e-21, 0.19751311570931906, 0.3818950368204676, 0.8173267884868272, 0.23809798085689526, 0.32602568873054283, -1.4956180716991734e-22, 1.5702228908505302e-21, -9.836010417912434e-22, -2.843786750513666e-21, 0.7775443540978032, 1.0895914503434282e-21, 1.3868528883849717e-21, 0.28867757701213453, -1.5194640360610535e-21, -1.1268153586575133e-21, 4.898250350410614e-22, -5.691690688689662e-22, 0.09957278034617512, 0.09147345538850993, 1.0, -7.856088836468182e-22, -2.9122297163612315e-22, 0.2264431864607574, 0.1429907888893215, 9.075682114908828e-22, 1.0, -8.706657967549065e-22, 0.5431501796588, 3.425181729369703e-22, 0.05973420975777847, 4.248291282562093e-21, 0.21128724240699553, -2.067065639889572e-21, 1.1077799554009854e-21, -3.213582479858265e-22, 1.6376051861804975e-21, 1.0058282419300883e-21, -4.581859854985512e-22, 0.7856832609639447, -5.2254418894831725e-22, 4.7024942799603785e-22, 0.43972597453100204, -1.